In [1]:
%pip show git+https://github.com/Oriolrt/GABDConnect ucimlrepo

Name: ucimlrepo
Version: 0.0.7
Summary: Package to easily import datasets from the UC Irvine Machine Learning Repository into scripts and notebooks.
Home-page: https://github.com/uci-ml-repo/ucimlrepo
Author: Philip Truong
Author-email: Philip Truong <ucirepository@gmail.com>
License: 
Location: c:\Users\Pau\Desktop\SpamDisc\venv\Lib\site-packages
Requires: certifi, pandas
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [1]:
# @title Install or Update GABDConnect Libraries



%pip install ipywidgets -q
%pip install ipython_genutils -q
%pip install ipykernel ipywidgets -q


import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

def install_packages(b):
    """
    Function to install or update necessary Python packages.

    Parameters:
    - b: Button object. Click event handler.
    """
    clear_output(wait=True)
    %pip install --upgrade pip -q  
    %pip install  git+https://github.com/Oriolrt/GABDConnect ucimlrepo -q  
    %pip install html2text -q  
    print("Libraries have been downloaded or updated.")

install_packages(True)



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Libraries have been downloaded or updated.


In [2]:
# @title  Fem els imports de les llibreries

from GABDConnect.oracleConnection import oracleConnection as orcl
import getpass
import logging

In [3]:
import os


# Inicialitzem el diccionari amb les dades de connexió SSH per fer el tunel
SSH_USER = "student"
SSH_TUNNEL = "dcccluster.uab.cat"

PORT = "8195"
PWD = "vj2wkFAmstop6"
hostname= "oracle-1.grup03.gabd"


# Dades per fer el tunel SSH
ssh_server = {'ssh': SSH_TUNNEL , 'user': SSH_USER,
                'pwd': PWD, 'port': PORT} if SSH_TUNNEL is not None else None

# Dades de connexió a Oracle
user = '"gestorUCI"'  # @param {type:"string"}   Segons s'indica a l'enunciat de la pràctica
oracle_pwd = "gestor_password"  # @param {type:"string"}  Poseu aqui la contrasenya que hagueu assignat a l'usuari gestorUCI
serviceName="FREEPDB1" # @param {type:"string"}

EXERCICI INSERCIO DE DADES

In [6]:
from ucimlrepo import fetch_ucirepo
import oracledb
import numpy as np
import json


def exists(dbConn, dataset):
  """

  :param dataset: name of the dataset
  :return: True if  the dataset is already inserted in the DB
  """

  query = f"select *  from dataset where  name = '{dataset}'"



  return dbConn.cursor().execute(query).fetchone() is not None
  # return False


def insertVectorDataset(dbConn, nameDataset,  *args, **kwargs):
  """
      Inserts the contents stored in fileName into de DB

      Params:
      :param dbConn: handle to an active (and open) connexion to Oracle DB.
      :param nameDataset: name of the dataset to be inserted into the database.
      :return: Boolean value whether data have properly been inserted into de database
  """
  
  # Si les dades estan inserides no caldrà fer res
  if exists(dbConn,nameDataset):
    return True
  print("Fetching repository:" + nameDataset)

  dataset = fetch_ucirepo(name=nameDataset)
  print(type(dataset))

  
  for row in dataset:
     
     for rowNest in dataset[row]:
       print("{"+ row+": " + rowNest +"}")
  
  #SAMPLES SAMPLES SAMPLES

  metadata = dataset.metadata
  repository_url = metadata["repository_url"]
  feat_size = metadata["num_features"]
  num_classes = len(set(dataset.data.targets))

  metadata_json = json.dumps({
        "repository_url": repository_url,
        "description": metadata.get("abstract", ""),
        "area": metadata.get("area", ""),
        "tasks": metadata.get("tasks", []),
        "num_instances": metadata.get("num_instances", 0),
        "feature_types": metadata.get("feature_types", []),
        "demographics": metadata.get("demographics", []),
        "has_missing_values": metadata.get("has_missing_values", False),
        "year_of_dataset_creation": metadata.get("year_of_dataset_creation", None),
        "last_updated": metadata.get("last_updated", None),
        "dataset_doi": metadata.get("dataset_doi", None),
        "creators": metadata.get("creators", []),
        "intro_paper": metadata.get("intro_paper", None),
        "additional_info": metadata.get("additional_info", None),
    })
  
  #dataframe
  
  

  cursor = dbConn.cursor()
  queryDataset = "INSERT INTO Dataset (id, name, feat_size, numclasses, info) VALUES (:1, :2, :3, :4, :5)"
  cursor.prepare(queryDataset)
  
  
  print("INSERT: name :"+ nameDataset + "|  feat_size: " + str(feat_size) + "| num class:"+ str(num_classes) )
  
  parametersDS = [metadata.uci_id , metadata["name"], len(dataset.data.features), num_classes, str(metadata_json)]
  cursor.execute(queryDataset, parametersDS)
  #---------------------SAMPLES
  querySamples = "INSERT INTO Samples (id_dataset, id, features, label) VALUES (:1, :2, :3, :4)"
  cursor.prepare(querySamples)  
 
  
  targets = np.array(dataset['data']['targets'])
  for index, row in enumerate(targets):
    
    vFeature = '[' + ','.join(map(str, dataset.data.features.iloc[index].tolist())) + ']'

    label = str(row).replace("['", "").replace("']", "")
    print("Row " + str(index) + ":  "+ vFeature + "   Class:" + label) 

    parametersSA = [str(metadata.uci_id) , str(index) , vFeature, label]
    cursor.execute(querySamples, parametersSA)
  
  ###################UCI_ID####################
  
  
  #cursor.execute(querySamples, parametersSA)

  
  
  
  
  


  try:
    dbConn.commit()
    return True
  except:
    return False


["Iris", "Breast Cancer Wisconsin (Diagnostic)", "Ionosphere", "Letter Recognition"]


In [7]:
# Cridem el constructor i obrim la connexió
db = orcl(user=user, passwd=oracle_pwd, hostname=hostname, ssh=ssh_server,serviceName=serviceName)

try:
  if not db.isStarted:
    db.open()
except Exception as e:
  print(e)


In [7]:
datasets = fetch_ucirepo(name="Iris")

In [9]:


#datasets['data']['targets']
datasets['data']['features'][:]
#datasets['variables']['role']
#datasets['metadata']
#datasets['variables']
insertVectorDataset(db, "Iris")
targets = np.array(datasets['data']['targets'])
for index, row in enumerate(targets):
    
    vFeature = '[' + ','.join(map(str, datasets.data.features.iloc[index].tolist())) + ']'
    print("Row " + str(index) + ":  "+ vFeature)
    



Row 0:  [5.1,3.5,1.4,0.2]
Row 1:  [4.9,3.0,1.4,0.2]
Row 2:  [4.7,3.2,1.3,0.2]
Row 3:  [4.6,3.1,1.5,0.2]
Row 4:  [5.0,3.6,1.4,0.2]
Row 5:  [5.4,3.9,1.7,0.4]
Row 6:  [4.6,3.4,1.4,0.3]
Row 7:  [5.0,3.4,1.5,0.2]
Row 8:  [4.4,2.9,1.4,0.2]
Row 9:  [4.9,3.1,1.5,0.1]
Row 10:  [5.4,3.7,1.5,0.2]
Row 11:  [4.8,3.4,1.6,0.2]
Row 12:  [4.8,3.0,1.4,0.1]
Row 13:  [4.3,3.0,1.1,0.1]
Row 14:  [5.8,4.0,1.2,0.2]
Row 15:  [5.7,4.4,1.5,0.4]
Row 16:  [5.4,3.9,1.3,0.4]
Row 17:  [5.1,3.5,1.4,0.3]
Row 18:  [5.7,3.8,1.7,0.3]
Row 19:  [5.1,3.8,1.5,0.3]
Row 20:  [5.4,3.4,1.7,0.2]
Row 21:  [5.1,3.7,1.5,0.4]
Row 22:  [4.6,3.6,1.0,0.2]
Row 23:  [5.1,3.3,1.7,0.5]
Row 24:  [4.8,3.4,1.9,0.2]
Row 25:  [5.0,3.0,1.6,0.2]
Row 26:  [5.0,3.4,1.6,0.4]
Row 27:  [5.2,3.5,1.5,0.2]
Row 28:  [5.2,3.4,1.4,0.2]
Row 29:  [4.7,3.2,1.6,0.2]
Row 30:  [4.8,3.1,1.6,0.2]
Row 31:  [5.4,3.4,1.5,0.4]
Row 32:  [5.2,4.1,1.5,0.1]
Row 33:  [5.5,4.2,1.4,0.2]
Row 34:  [4.9,3.1,1.5,0.1]
Row 35:  [5.0,3.2,1.2,0.2]
Row 36:  [5.5,3.5,1.3,0.2]
Row 37:  [4

In [8]:
for name in ["Iris", "Breast Cancer Wisconsin (Diagnostic)", "Ionosphere", "Letter Recognition"]:
  res = insertVectorDataset(db, name )

  if res:
    logging.warning(f"Dades a {name} carregades correctament.")
  else:
    logging.warning(f"Les Dades a {name} no s'han inserit correctament.")

Fetching repository:Iris
<class 'ucimlrepo.dotdict.dotdict'>
{data: ids}
{data: features}
{data: targets}
{data: original}
{data: headers}
{metadata: uci_id}
{metadata: name}
{metadata: repository_url}
{metadata: data_url}
{metadata: abstract}
{metadata: area}
{metadata: tasks}
{metadata: characteristics}
{metadata: num_instances}
{metadata: num_features}
{metadata: feature_types}
{metadata: demographics}
{metadata: target_col}
{metadata: index_col}
{metadata: has_missing_values}
{metadata: missing_values_symbol}
{metadata: year_of_dataset_creation}
{metadata: last_updated}
{metadata: dataset_doi}
{metadata: creators}
{metadata: intro_paper}
{metadata: additional_info}
{variables: name}
{variables: role}
{variables: type}
{variables: demographic}
{variables: description}
{variables: units}
{variables: missing_values}
INSERT: name :Iris|  feat_size: 4| num class:1
Row 0:  [5.1,3.5,1.4,0.2]   Class:Iris-setosa
Row 1:  [4.9,3.0,1.4,0.2]   Class:Iris-setosa
Row 2:  [4.7,3.2,1.3,0.2]   Clas

Row 147:  [6.5,3.0,5.2,2.0]   Class:Iris-virginica
Row 148:  [6.2,3.4,5.4,2.3]   Class:Iris-virginica
Row 149:  [5.9,3.0,5.1,1.8]   Class:Iris-virginica
Fetching repository:Breast Cancer Wisconsin (Diagnostic)
<class 'ucimlrepo.dotdict.dotdict'>
{data: ids}
{data: features}
{data: targets}
{data: original}
{data: headers}
{metadata: uci_id}
{metadata: name}
{metadata: repository_url}
{metadata: data_url}
{metadata: abstract}
{metadata: area}
{metadata: tasks}
{metadata: characteristics}
{metadata: num_instances}
{metadata: num_features}
{metadata: feature_types}
{metadata: demographics}
{metadata: target_col}
{metadata: index_col}
{metadata: has_missing_values}
{metadata: missing_values_symbol}
{metadata: year_of_dataset_creation}
{metadata: last_updated}
{metadata: dataset_doi}
{metadata: creators}
{metadata: intro_paper}
{metadata: additional_info}
{variables: name}
{variables: role}
{variables: type}
{variables: demographic}
{variables: description}
{variables: units}
{variables: mi

Row 553:  [9.333,21.94,59.01,264.0,0.0924,0.05605,0.03996,0.01282,0.1692,0.06576,0.3013,1.879,2.121,17.86,0.01094,0.01834,0.03996,0.01282,0.03759,0.004623,9.845,25.05,62.86,295.8,0.1103,0.08298,0.07993,0.02564,0.2435,0.07393]   Class:B
Row 554:  [12.88,28.92,82.5,514.3,0.08123,0.05824,0.06195,0.02343,0.1566,0.05708,0.2116,1.36,1.502,16.83,0.008412,0.02153,0.03898,0.00762,0.01695,0.002801,13.89,35.74,88.84,595.7,0.1227,0.162,0.2439,0.06493,0.2372,0.07242]   Class:B
Row 555:  [10.29,27.61,65.67,321.4,0.0903,0.07658,0.05999,0.02738,0.1593,0.06127,0.2199,2.239,1.437,14.46,0.01205,0.02736,0.04804,0.01721,0.01843,0.004938,10.84,34.91,69.57,357.6,0.1384,0.171,0.2,0.09127,0.2226,0.08283]   Class:B
Row 556:  [10.16,19.59,64.73,311.7,0.1003,0.07504,0.005025,0.01116,0.1791,0.06331,0.2441,2.09,1.648,16.8,0.01291,0.02222,0.004174,0.007082,0.02572,0.002278,10.65,22.88,67.88,347.3,0.1265,0.12,0.01005,0.02232,0.2262,0.06742]   Class:B
Row 557:  [9.423,27.88,59.26,271.3,0.08123,0.04971,0.0,0.0,0.1742,0

Row 339:  [1.0,0.0,0.65845,0.43617,0.44681,0.74804,0.05319,0.85106,-0.32027,0.82139,-0.68253,0.52408,-0.84211,0.07111,-0.82811,-0.28723,-0.47032,-0.71725,-0.04759,-0.86002,0.23292,-0.76316,0.56663,-0.52128,0.743,-0.18645,0.74758,0.23713,0.45185,0.59071,0.20549,0.76764,-0.18533,0.74356]   Class:g
Row 340:  [1.0,0.0,0.19466,0.05725,0.04198,0.25191,-0.10557,0.48866,-0.18321,-0.18321,-0.41985,0.06107,-0.4542,0.0916,-0.16412,-0.30534,-0.10305,-0.39695,0.18702,-0.17557,0.34012,-0.11953,0.28626,-0.16031,0.21645,0.24692,0.03913,0.31092,-0.03817,0.26336,-0.16794,0.16794,-0.30153,-0.33588]   Class:g
Row 341:  [1.0,0.0,0.98002,0.00075,1.0,0.0,0.98982,-0.00075,0.94721,0.02394,0.977,0.0213,0.97888,0.03073,0.9917,0.02338,0.93929,0.05713,0.93552,0.05279,0.97738,0.05524,1.0,0.06241,0.94155,0.08107,0.96709,0.07255,0.95701,0.08088,0.9819,0.08126,0.97247,0.08616]   Class:g
Row 342:  [1.0,0.0,0.82254,-0.07572,0.80462,0.00231,0.87514,-0.01214,0.86821,-0.07514,0.72832,-0.11734,0.84624,0.05029,0.83121,-0.073

Row 19998:  [2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8]   Class:S
Row 19999:  [4,9,6,6,2,9,5,3,1,8,1,8,2,7,2,8]   Class:A


In [ ]:
if  db.isStarted:
  db.close()